# Install and Import all Libraries

In [1]:
# installing pathway bokeh
!pip install pathway bokeh --quiet

In [2]:
# suppress specific user warnings from Bokeh

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [3]:
# core Libraries

import numpy as np
import pandas as pd
import datetime
from datetime import datetime

In [4]:
# Pathway & Bokeh

import pathway as pw
import bokeh.plotting
import panel as pn

# Load and Preprocess Parking Dataset

In [5]:
# load the dataset

df = pd.read_csv('/content/dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


## Checking missing values

In [6]:
# check for missing values

df.isna().sum()

,0
ID,0
SystemCodeNumber,0
Capacity,0
Latitude,0
Longitude,0
Occupancy,0
VehicleType,0
TrafficConditionNearby,0
QueueLength,0
IsSpecialDay,0


In [7]:
# combine date and time columns into a single datetime column

df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

## Sort record by timestamps

In [8]:
# sort records by timestamp

df = df.sort_values('Timestamp').reset_index(drop=True)

## Computing Occupancy Rates

In [9]:
# compute occupancy rate

df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]

In [10]:
# preview processed data

df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,OccupancyRate
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.105719
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.197500
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.384279
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.513402
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.319792


# Export Streamable Subset for Pathway

In [11]:
# save selected columns to a CSV to simulate a data stream

df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "OccupancyRate"]].to_csv("parking_stream.csv", index=False)

# Define Schema and Load Streaming Data with Pathway

In [12]:
# define schema for Pathway ingestion

class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    OccupancyRate: float

In [13]:
# load data with simulated stream rate

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=100)

# Convert Time Columns and Create Daily Lot Identifiers

In [14]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d"),
    OccupancyRate = data.Occupancy / data.Capacity,
    day_id = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%d") + "_" + data.SystemCodeNumber,
)

In [15]:
# ccreate lookup table of unique lot-day combinations

id_lookup = (
    data_with_time
    .select(pw.this.day_id, pw.this.SystemCodeNumber)
    .groupby(pw.this.day_id)
    .reduce(
        day_id = pw.reducers.min(pw.this.day_id),
        SystemCodeNumber = pw.reducers.min(pw.this.SystemCodeNumber)
    )
)

# Aggregate Daily Occupancy Stats and Compute Dynamic Price

In [19]:
from datetime import timedelta

# daily rolling window aggregation

reduced = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day_id,
        window=pw.temporal.tumbling(timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.OccupancyRate),
        occ_min = pw.reducers.min(pw.this.OccupancyRate),
        cap = pw.reducers.max(pw.this.Capacity),
        day_id = pw.reducers.min(pw.this.day_id),
        lot = pw.reducers.min(pw.this.SystemCodeNumber)
    )
)

# pricing formula: base price + scaled volatility

delta_window = reduced.with_columns(
    price = 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
)

# Live Streaming Plot using Panel and Bokeh

In [20]:
pn.extension()

# define interactive Bokeh plot


def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )

    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.scatter("t", "price", source=source, size=6, color="red", marker="circle")

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"
    return fig

# launch panel dashboard

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()
viz.servable()

Column
    [0] Row
        [0] Markdown(str)
        [1] TooltipIcon(value='Table depends o...)
    [1] Bokeh(figure)

# Export Real-Time Price Output to JSON Lines

In [21]:
# write output to file for post-processing and visualization

pw.io.jsonlines.write(
    delta_window.select(
        pw.this.t,
        pw.this.lot,
        pw.this.price
    ),
    "pricing_output.jsonl"
)

# Run Pathway app

pw.run()


/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


Output()

# Post-Run Visualization by Lot (Multiple Plots)

In [22]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

output_notebook()

In [23]:
# read saved prices

df_price = pd.read_json("pricing_output.jsonl", lines=True)
df_price["t"] = pd.to_datetime(df_price["t"])
df_price["lot"] = df_price["lot"].astype(str)

plots = []
lots = sorted(df_price["lot"].unique())
palette = Category10[10] + Category10[4]

for i, lot in enumerate(lots):
    df_lot = df_price[df_price["lot"] == lot].copy()
    source = ColumnDataSource(df_lot[["t", "price", "lot"]])

    fig = figure(
        height=300,
        width=800,
        title=f"Price Trend - Lot {lot}",
        x_axis_type="datetime",
        tools="pan,wheel_zoom,box_zoom,reset"
    )

    fig.line("t", "price", source=source, color=palette[i % len(palette)], line_width=2)
    fig.scatter("t", "price", source=source, size=5, marker="circle", color="black")

    hover = HoverTool(
        tooltips=[
            ("Lot", "@lot"),
            ("Time", "@t{%F %T}"),
            ("Price", "@price{$0.00}")
        ],
        formatters={"@t": "datetime"},
        mode="vline"
    )
    fig.add_tools(hover)

    fig.xaxis.axis_label = "Time"
    fig.yaxis.axis_label = "Price ($)"

    plots.append(fig)

show(column(*plots))

# Final Summary Plot: All Lots Combined

In [24]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

output_notebook()

# normalize datetime precision

df = pd.read_json("pricing_output.jsonl", lines=True)
df["t"] = pd.to_datetime(df["t"]).dt.tz_localize(None)
df["t"] = df["t"].astype("datetime64[ms]")
df["lot"] = df["lot"].astype(str)

In [25]:
# create combined figure

fig = figure(
    height=500,
    width=900,
    title="Daily Pricing Across Parking Lots",
    x_axis_type="datetime"
)

palette = Category10[10] + Category10[4]
lots = sorted(df["lot"].unique())

for i, lot in enumerate(lots):
    df_lot = df[df["lot"] == lot].copy()
    source = ColumnDataSource(df_lot[["t", "price", "lot"]])

    fig.line("t", "price", source=source, legend_label=f"Lot {lot}",
             color=palette[i % len(palette)], line_width=2)

In [26]:
#add interactive hover tool

hover = HoverTool(
    tooltips=[
        ("Lot", "@lot"),
        ("Time", "@t{%F %T}"),
        ("Price", "@price{$0.00}")
    ],
    formatters={"@t": "datetime"},
    mode="vline"
)
fig.add_tools(hover)

fig.xaxis.axis_label = "Time"
fig.yaxis.axis_label = "Price ($)"
fig.legend.label_text_font_size = "8pt"

In [27]:
#show plot

show(fig)